In [ ]:
import os
import json
from math import sin, cos, pi, atan2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset
from shapely.geometry import Point, LineString

csv_path = "path_to_diningroom_clusters.csv"
df = pd.read_csv(csv_path)

print("Column names in the csv file:")
print(df.columns.tolist())

label_cols = []
for col in df.columns:
    if "_Type" in col:
        label_cols.append(col)

unique_labels = set()
for col in label_cols:
    for value in df[col].dropna():
        unique_labels.add(value)

label_list = sorted(unique_labels)
label_to_id = {}
for i, label in enumerate(label_list):
    label_to_id[label] = i

print("Total unique labels:", len(label_to_id))
print(label_to_id)

print(os.listdir("path_to_diningroom_heatmaps"))
# ------------------------------------------------------------------

IMAGE_SIZE = 256

def norm_to_px(v_norm: float, img_size: int = IMAGE_SIZE) -> int:
    return int((v_norm + 5.0) / 10.0 * img_size)

scene_to_objects = {}

# ------------------------------------------------------------------
for _, row in df.iterrows():
    sid = row["Scene_ID"]
    objs = []
    for i in range(6):
        label     = row[f"Cluster_{i}_Type"]
        x_norm    = row[f"Cluster_{i}_Centroid_X"]
        y_norm    = row[f"Cluster_{i}_Centroid_Y"]
        theta_raw = row[f"Cluster_{i}_Orientations"]

        if pd.isna(label) or pd.isna(x_norm) or pd.isna(y_norm) or pd.isna(theta_raw):
            continue

        x_px = norm_to_px(x_norm)
        y_px = norm_to_px(y_norm)

        theta = None
        if isinstance(theta_raw, str) and ":" in theta_raw:
            for chunk in theta_raw.split(","):
                parts = chunk.strip().split(":")
                if len(parts) >= 3:
                    _, lbl, val = parts
                    if lbl.strip() == label.strip():
                        try:
                            theta = float(val.strip().replace("°", ""))
                            break
                        except ValueError:
                            pass
        if theta is None:
            try:
                theta = float(str(theta_raw).replace("°", ""))
            except ValueError:
                continue

        objs.append({
            "x":     x_px,
            "y":     y_px,
            "theta": theta,
            "label": label
        })


    if objs:
        scene_to_objects[sid] = objs

print("Parsed scene_to_objects:", len(scene_to_objects))

with open("path_to_diningroom_wall_data.json") as f:
    wall_data = json.load(f)

def polyline_to_dist(poly, H, W):
    canvas = torch.zeros((H, W))
    pts = torch.tensor(poly).long()
    pts[:, 0] = pts[:, 0].clamp(0, W - 1)
    pts[:, 1] = pts[:, 1].clamp(0, H - 1)
    canvas[pts[:,1], pts[:,0]] = 1.0

    grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing="ij")
    coords = torch.stack([grid_x, grid_y], dim=-1).reshape(-1, 2).float()
    wall_pts = torch.nonzero(canvas).float()

    dist = torch.cdist(coords, wall_pts).min(dim=1)[0].reshape(H, W)
    return dist / dist.max()


bad_scene = "00154c06-2ee2-408a-9664-b8fd74742897_DiningRoom-17932"
wall_data = [entry for entry in wall_data if entry["scene_id"] != bad_scene]

heatmap_root = Path("path_to_diningroom_heatmaps")
heatmap_labels = sorted([p.name for p in heatmap_root.iterdir() if p.is_dir()])
print("Heatmap labels:", len(heatmap_labels))

class DiningRoomDatasetWithTheta(Dataset):
    def __init__(self, wall_entries, heatmap_folder, heatmap_labels, scene_objects, label_to_id_map):
        self.entries = wall_entries
        self.heatmap_root = Path(heatmap_folder)
        self.labels = heatmap_labels
        self.scene_objects = scene_objects
        self.label_to_id = label_to_id_map

    def __len__(self):
        return len(self.entries)

    def load_mask(self, image_path):
        image = Image.open(image_path).convert("L")
        return torch.from_numpy(np.array(image) > 0).float()

    def load_heatmaps(self, scene_id, height, width):
        heatmap_stack = []
        for label in self.labels:
            heatmap_path = self.heatmap_root / label / f"{scene_id}.png"
            if heatmap_path.exists():
                image = Image.open(heatmap_path).convert("L").resize((width, height))
                heatmap = np.array(image) / 255.0
                heatmap_stack.append(torch.from_numpy(heatmap).float())
            else:
                heatmap_stack.append(torch.zeros((height, width)))
        return torch.stack(heatmap_stack)

    def __getitem__(self, index):
        item = self.entries[index]
        scene_id = item["scene_id"]
        mask_path = item["room_mask"]
        wall_polyline = item["wall_polyline"]

        mask = self.load_mask(mask_path)
        height, width = mask.shape

        wall_distance = polyline_to_dist(wall_polyline, height, width)
        heatmaps = self.load_heatmaps(scene_id, height, width)

        input_tensor = torch.cat([mask.unsqueeze(0), wall_distance.unsqueeze(0), heatmaps], dim=0)

        object_list = self.scene_objects.get(scene_id, [])

        ground_truth = []

        for obj in object_list:
            x = obj["x"]
            y = obj["y"]
            label = obj["label"]

            angle_rad = float(obj["theta"])
            label_index = self.label_to_id.get(label, -1)

            if label_index >= 0:
                ground_truth.append([x, y, angle_rad, label_index])

        target_tensor = torch.tensor(ground_truth, dtype=torch.float32)

        return input_tensor, target_tensor, scene_id


ds = DiningRoomDatasetWithTheta(
    wall_entries=wall_data,
    heatmap_folder="path_to_diningroom_heatmaps",
    heatmap_labels=heatmap_labels,
    scene_objects=scene_to_objects,
    label_to_id_map=label_to_id
)

x, objs, sid = ds[0]
print("Scene ID:", sid)
print("Tensor shape:", x.shape)  
print("Ground truth shape:", objs.shape)  
print("Total scenes in the dataset:", len(ds))

def rule_penality_engine(objs, polyline, label_id_to_name):
    wall = LineString(polyline)
    rules = {f"Rule {i}": 0.0 for i in range(1, 16)}

    def angle_difference(a1, a2):
        return min(abs(a1 - a2), 2 * pi - abs(a1 - a2))

    obj_by_label = {}

    for index, (x, y, angle, label_id) in enumerate(objs):
        label = label_id_to_name.get(int(label_id), f"#{int(label_id)}")
        obj_by_label.setdefault(label, []).append((x, y, angle))

        point = Point(float(x), float(y))
        dist_to_wall = point.distance(wall)

        # Rule 1: Dining chair too close to wall
        if label == "dining_chair" and dist_to_wall < 30:
            rules["Rule 1"] += (30 - dist_to_wall) ** 2
        elif dist_to_wall < 100:
            rules["Rule 1"] += (100 - dist_to_wall) * 0.5

        # Rule 2: Sofa facing wall too closely
        if label == "sofa":
            dx = cos(angle)
            dy = sin(angle)
            ahead = Point(x + dx * 10, y + dy * 10)
            if ahead.distance(wall) < point.distance(wall):
                if dist_to_wall < 50:
                    rules["Rule 2"] += (50 - dist_to_wall) ** 2
                elif dist_to_wall < 100:
                    rules["Rule 2"] += (100 - dist_to_wall) * 0.5

        # Rule 7: Chair facing wall
        if label in ["dining_chair", "chair"] and dist_to_wall < 40:
            dx = cos(angle)
            dy = sin(angle)
            ahead = Point(x + dx * 10, y + dy * 10)
            if ahead.distance(wall) < point.distance(wall):
                if dist_to_wall < 30:
                    rules["Rule 7"] += (30 - dist_to_wall) ** 2
                elif dist_to_wall < 70:
                    rules["Rule 7"] += (70 - dist_to_wall) * 0.5

        # Rule 8: Object facing back wall
        if dist_to_wall < 40:
            back_diff = abs(angle - pi)
            if back_diff < 0.3:
                rules["Rule 8"] += 10
            elif back_diff < 0.6:
                rules["Rule 8"] += 5

    # Rule 15: Blocking center passage with large object
    for x, y, _, label_id in objs:
        label = label_id_to_name.get(int(label_id), "")
        dist = np.linalg.norm([x - 128, y - 128])
        if dist < 60:
            if label in ["sofa", "dining_table", "console_table", "Outlier_console_table"]:
                rules["Rule 15"] += (60 - dist) * 0.5
            elif label in ["chair", "dining_chair"]:
                rules["Rule 15"] += (60 - dist) * 0.2

    # Rule 3: Objects overlapping too closely
    for i in range(len(objs)):
        for j in range(i + 1, len(objs)):
            xi, yi, _, lid_i = objs[i]
            xj, yj, _, lid_j = objs[j]
            li = label_id_to_name.get(int(lid_i), "")
            lj = label_id_to_name.get(int(lid_j), "")
            skip = [
                ("dining_chair", "dining_table"),
                ("dining_chair", "dining_chair"),
                ("chair", "table"),
                ("chair", "coffee_table"),
                ("sofa", "coffee_table")
            ]
            if (li, lj) in skip or (lj, li) in skip:
                continue
            d = np.linalg.norm([xi - xj, yi - yj])
            if d < 30:
                rules["Rule 3"] += (30 - d) ** 2

    # Rule 4: Dining chair too far from dining table
    if "dining_chair" in obj_by_label and "dining_table" in obj_by_label:
        tx, ty, _ = obj_by_label["dining_table"][0]
        for cx, cy, _ in obj_by_label["dining_chair"]:
            d = np.linalg.norm([cx - tx, cy - ty])
            if d > 80:
                rules["Rule 4"] += (d - 80) ** 2

    # Rule 5: Dining chair behind the dining table
    if "dining_chair" in obj_by_label and "dining_table" in obj_by_label:
        _, table_y, _ = obj_by_label["dining_table"][0]
        for cx, cy, _ in obj_by_label["dining_chair"]:
            if cy > table_y + 40:
                rules["Rule 5"] += 20

    # Rule 6: Sofa not facing toward TV
    if "sofa" in obj_by_label and "tv" in obj_by_label:
        sx, sy, sa = obj_by_label["sofa"][0]
        tx, ty, _ = obj_by_label["tv"][0]
        view_angle = atan2(ty - sy, tx - sx)
        diff = angle_difference(view_angle, sa)
        if diff > 0.6:
            rules["Rule 6"] += diff * 30

    # Rule 9: Reward layout where sofa is away from wall and TV is near it
    if "sofa" in obj_by_label and "tv" in obj_by_label:
        sx, sy, _ = obj_by_label["sofa"][0]
        tx, ty, _ = obj_by_label["tv"][0]
        if Point(sx, sy).distance(wall) > 80 and Point(tx, ty).distance(wall) < 20:
            rules["Rule 9"] -= 10

    # Rule 10: Console table facing open space not wall
    if "Outlier_console_table" in obj_by_label:
        for x, y, angle in obj_by_label["Outlier_console_table"]:
            dx = cos(angle)
            dy = sin(angle)
            ahead = Point(x + dx * 20, y + dy * 20)
            if ahead.distance(wall) > Point(x, y).distance(wall):
                rules["Rule 10"] += 20
                
    # Rule 11 ─ Object far from all others
    if len(objs) > 1:
        for i, (xi, yi, _, _) in enumerate(objs):
            nearest = min(
                np.linalg.norm([xi - xj, yi - yj])
                for j, (xj, yj, _, _) in enumerate(objs) if i != j
            )
            if nearest > 80:
                rules["Rule 11"] += (nearest - 80) * 0.4

    # Rule 12: Dining table not aligned with room axes
    if "dining_table" in obj_by_label:
        _, _, angle = obj_by_label["dining_table"][0]
        aligned = min(abs(angle), abs(angle - pi / 2), abs(angle - pi), abs(angle - 3 * pi / 2))
        if aligned > 0.3:
            rules["Rule 12"] += aligned * 20

    # Rule 13: Too many objects in one corner
    corners = 0
    for x, y, _, _ in objs:
        if (x < 60 and y < 60) or (x > 196 and y < 60) or (x < 60 and y > 196) or (x > 196 and y > 196):
            corners += 1
    if corners >= 3:
        rules["Rule 13"] += corners * 5

    # Rule 14: Center of room is empty
    center = 0
    for x, y, _, _ in objs:
        if 80 < x < 176 and 80 < y < 176:
            center += 1
    if center == 0:
        rules["Rule 14"] += 30
    elif center == 1:
        rules["Rule 14"] += 10

    return rules

In [ ]:

import random, math, json, os, numpy as np, torch
from torch import nn
from pathlib import Path
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv



In [ ]:
# converting labels to ids
id_to_label = {v: k for k, v in label_to_id.items()}
print(f"Dataset size: {len(ds)}   |   #labels: {len(id_to_label)}")


In [ ]:
def make_empty_sample(idx):
    inp, gt, scene_id = ds[idx] 
    mask       = inp[0].numpy()
    wall_dist  = inp[1].numpy()
    
    entry      = next(e for e in wall_data if e["scene_id"] == scene_id)
    polyline   = entry["wall_polyline"]

    cluster_seq = [int(lid) for *_, lid in gt]
    random.shuffle(cluster_seq)
    
    return mask, wall_dist, polyline, cluster_seq, scene_id


In [ ]:
class RoomPlacementEnv(gym.Env):
    GRID         = 16
    IMG_H = IMG_W = 256
    MAX_STEPS    = 10

    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode=None, sample_idx=None):
        super().__init__()

        self.observation_space = spaces.Box(
            low=0, high=1, shape=(3, self.IMG_H, self.IMG_W), dtype=np.float32
        )
        self.action_space = spaces.Discrete(self.GRID ** 2)

        self.render_mode = render_mode
        self.fixed_idx   = sample_idx
        self.state       = None
        self.steps       = 0

        self._valid_xy   = []
        self._index_map  = []

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.steps = 0

        idx = self.fixed_idx if self.fixed_idx is not None else random.randint(0, len(ds)-1)
        mask, wall, poly, cluster_seq, scene_id = make_empty_sample(idx)


        self._valid_xy = []
        idx_to_valid   = [-1]* (self.GRID**2)

        for gi in range(self.GRID**2):
            gx, gy = gi % self.GRID, gi // self.GRID
            x = int((gx+0.5)*self.IMG_W/self.GRID)
            y = int((gy+0.5)*self.IMG_H/self.GRID)
            if mask[y, x] > 0:
                idx_to_valid[gi] = len(self._valid_xy)
                self._valid_xy.append((x, y))

        for gi in range(self.GRID**2):
            if idx_to_valid[gi] != -1:
                continue
            gx, gy = gi % self.GRID, gi // self.GRID
            x = int((gx+0.5)*self.IMG_W/self.GRID)
            y = int((gy+0.5)*self.IMG_H/self.GRID)

            nearest = min(range(len(self._valid_xy)),
                          key=lambda k: (self._valid_xy[k][0]-x)**2 + (self._valid_xy[k][1]-y)**2)
            idx_to_valid[gi] = nearest
        self._index_map = idx_to_valid

        self.state = {
            "mask":      mask,
            "wall":      wall,
            "poly":      poly,
            "scene_id":  scene_id,
            "occupancy": np.zeros_like(mask),
            "placed":    [],
            "remaining": cluster_seq[:self.MAX_STEPS],
        }
        return self._obs(), {}

    def _decode(self, a: int):
        v_idx      = self._index_map[a]
        x, y       = self._valid_xy[v_idx]
        return x, y

    def _obs(self):
        occ = self.state["occupancy"]
        return np.stack([self.state["mask"], self.state["wall"], occ]).astype(np.float32)

    def step(self, action: int):
        self.steps += 1
        if not self.state["remaining"]:
            return self._obs(), 0.0, True, False, {}

        label_id = self.state["remaining"].pop(0)
        x, y     = self._decode(action)
        theta    = 0.0
        self.state["placed"].append((x, y, theta, label_id))
        self.state["occupancy"][y, x] = 1.0

        objs_arr  = np.array(self.state["placed"], dtype=np.float32)
        reward    = -sum(rule_penality_engine(objs_arr, self.state["poly"], id_to_label).values())

        done = (not self.state["remaining"]) or (self.steps >= self.MAX_STEPS)
        return self._obs(), reward, done, False, {}

    def render(self):
        if self.render_mode != "human":
            return
        import matplotlib.pyplot as plt
        plt.figure(figsize=(4,4))
        plt.imshow(self.state["mask"], cmap="gray")
        for (x,y,_,lid) in self.state["placed"]:
            plt.scatter([x],[y], s=30)
            plt.text(x, y, id_to_label[lid][:2], color="red", fontsize=6)
        plt.axis("off")
        plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def show_original(env):

    sid = env.state["scene_id"]
    entry = next(e for e in wall_data if e["scene_id"] == sid)
    img   = Image.open(entry["rendered_image"])

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.title(f"Original scene: {sid[:8]}…")
    plt.axis("off")
    plt.show()


In [ ]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class SimpleCNN(BaseFeaturesExtractor):
    def __init__(self, obs_space: spaces.Box, features_dim=256):
        super().__init__(obs_space, features_dim)
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(32,64,4,2,1),  nn.ReLU(),
            nn.Conv2d(64,128,4,2,1), nn.ReLU(),
            nn.Conv2d(128,256,4,2,1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256*16*16, features_dim), nn.ReLU()
        )
    def forward(self, x):
        return self.net(x)

# class ResNetFeatureExtractor(BaseFeaturesExtractor):
#     """
#     :param obs_space: Observation space (Box(C,H,W))
#     :param features_dim: Size of the output feature vector
#     :param pretrained: Use ImageNet-pretrained weights
#     """
#     def __init__(self,
#                  obs_space: spaces.Box,
#                  features_dim: int = 256,
#                  pretrained: bool = True):
#         super().__init__(obs_space, features_dim)

#         resnet = models.resnet18(pretrained=pretrained)
#         self.backbone = nn.Sequential(*list(resnet.children())[:-2])
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

#         backbone_out = resnet.inplanes
#         self.projection = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(backbone_out, features_dim),
#             nn.ReLU()
#         )

#         self._features_dim = features_dim

#     def forward(self, observations):
#         x = self.backbone(observations)
#         x = self.avgpool(x)
#         return self.projection(x)


In [ ]:
# dummy test
def make_env():
    return RoomPlacementEnv()

vec_env = DummyVecEnv([make_env])
obs = vec_env.reset()
print("Obs shape:", obs.shape, "|  random reward test:", vec_env.step([vec_env.action_space.sample()])[1])


In [ ]:
policy_kwargs = dict(features_extractor_class=SimpleCNN, features_extractor_kwargs=dict(features_dim=256))

model = PPO(
    "CnnPolicy",
    vec_env,
    learning_rate=3e-4,
    n_steps=1024,
    batch_size=64,
    n_epochs=25,
    gamma=0.99,
    policy_kwargs=policy_kwargs,
    verbose=1,
)

TIMESTEPS = 20000     
model.learn(total_timesteps=TIMESTEPS)
model.save("ppo_roomplacer_mvp_diningroom")
print("MVP training done")


In [ ]:

try:
    label_id_to_name
except NameError:
    label_id_to_name = {v: k for k, v in label_to_id.items()}


In [ ]:
import copy, numpy as np, matplotlib.pyplot as plt, pandas as pd, os

try:
    label_id_to_name
except NameError:
    label_id_to_name = {v: k for k, v in label_to_id.items()}

env_live = RoomPlacementEnv(render_mode="human")
obs_live, _ = env_live.reset()
print("Initial clusters in this room:", len(env_live.state["remaining"]))
print("New room loaded with", len(env_live.state["remaining"]), "objects to go.")

def rollout_candidate(env_base, num_to_place, seed=None):
    rng = np.random.RandomState(seed)
    env_cand = copy.deepcopy(env_base)
    obs = env_cand._obs()
    for _ in range(num_to_place):
        if not env_cand.state["remaining"]:
            break
        act, _ = model.predict(obs, deterministic=False)
        if rng.rand() < 0.1:
            act = env_cand.action_space.sample()
        obs, _, done, _, _ = env_cand.step(int(act))
        if done:
            break
    return env_cand

def total_penalty(env_):
    objs = np.array(env_.state["placed"], dtype=np.float32)
    pen  = rule_penality_engine(objs, env_.state["poly"], label_id_to_name)
    return sum(pen.values())

while True:
    try:
        k = int(input("\nHow many NEW objects should I try to place? (0 to quit) ➜ "))
    except ValueError:
        print("Please enter an integer."); continue
    if k <= 0:
        print("Stopping. Final layout shown above."); break
    if not env_live.state["remaining"]:
        print("All objects are already placed!"); break
    k = min(k, len(env_live.state["remaining"]))

    print(f"Generating 5 candidate layouts, each placing {k} objects …")
    candidates = [(total_penalty(env:=rollout_candidate(env_live, k, i)), env) for i in range(5)]
    best_pen, env_live = min(candidates, key=lambda x: x[0])
    obs_live = env_live._obs()

    print(f"Selected candidate with total penalty = {best_pen:.2f}")
    env_live.render()
    show_original(env_live)
    print(f"{len(env_live.state['remaining'])} objects still unplaced.")

    if input("Add more objects? [y/n] ➜ ").lower().strip() != "y":
        print("Interaction finished."); break

print("Saving final_layout_interactive.png & placement_log_interactive.csv …")
env_live.render()
plt.savefig("final_layout_interactive.png", bbox_inches="tight", dpi=200)

pd.DataFrame(env_live.state["placed"],
             columns=["x","y","theta","label_id"]
).to_csv("placement_log_interactive.csv", index=False)
print("Files saved in", os.getcwd())

print("Saving final_layout_interactive.png & placement_log_interactive.csv …")
env_live.render()
plt.savefig("final_layout_interactive.png", bbox_inches="tight", dpi=200)

placement_log = pd.DataFrame(
    env_live.state["placed"],
    columns=["x","y","theta","label_id"]
)
placement_log.to_csv("placement_log_interactive.csv", index=False)
print("Files saved in", os.getcwd())

print("\nFinal placement summary:")
for idx, (x, y, theta, label_id) in enumerate(env_live.state["placed"], start=1):
    label = label_id_to_name[label_id]
    print(f"{idx:2d}. {label:20s} @ (x={x:.1f}, y={y:.1f}), θ={theta:.2f} rad")



In [ ]:
model_path = "ppo_roomplacer_mvp_diningroom.zip"  
model = PPO.load(model_path)

env = RoomPlacementEnv(render_mode="human")
obs, _ = env.reset()

def compute_penalty(env_instance):
    arr = np.array(env_instance.state["placed"], dtype=np.float32)
    pen = rule_penality_engine(arr, env_instance.state["poly"], label_id_to_name)
    return sum(pen.values())

print(f"\nStarting interactive placement; {len(env.state['remaining'])} objects to place.")

while True:
    ans = input("\nPlace one more object? [y/n] ➜ ").strip().lower()
    if ans != "y":
        print("Stopping interactive placement.")
        break

    if not env.state["remaining"]:
        print("No more objects left to place.")
        break
    N_CANDIDATES = 100
    best_pen = float("inf")
    best_env = None

    print(f"Generating {N_CANDIDATES} placement candidates for the next object...")
    for seed in range(N_CANDIDATES):
        cand_env = copy.deepcopy(env)
        obs_cand = cand_env._obs()

        action, _ = model.predict(obs_cand, deterministic=False)
        if np.random.rand() < 0.1:
            action = cand_env.action_space.sample()

        obs_new, _, done, _, _ = cand_env.step(int(action))
        pen = compute_penalty(cand_env)

        if pen < best_pen:
            best_pen = pen
            best_env = cand_env

    env = best_env
    print(f"→ Chose placement with penalty = {best_pen:.2f}")
    env.render()
    print(f"{len(env.state['remaining'])} objects still unplaced.")

print("\nFinal layout:")
env.render()
show_original(env)
plt.savefig("final_layout_interactive.png", bbox_inches="tight", dpi=200)

placements = pd.DataFrame(env.state["placed"], columns=["x","y","theta","label_id"])
placements["label"] = placements["label_id"].map(label_id_to_name)
placements.to_csv("placement_log_interactive.csv", index=False)

print("\nPlacement log:")
print(placements)

print(f"\nFiles written to {os.getcwd()}:")
print("  - final_layout_interactive.png")
print("  - placement_log_interactive.csv")

In [ ]:
import pandas as pd

csv_path = "path_to_diningroom_clusters.csv"
df = pd.read_csv(csv_path)

for i in range(6):
    col = f"Cluster_{i}_Orientations"
    if col not in df.columns:
        continue

    print(f"\n=== {col} ===")
    nonnull = df[col].dropna()
    print("Raw sample values:", nonnull.head(10).tolist())
    try:
        nums = nonnull.astype(float)
        print(nums.describe())
    except ValueError:
        print("  (some entries aren’t plain floats; they’ll need string parsing)")
